In [1]:
import MySQLdb
from MySQLdb import OperationalError

In [35]:
db = MySQLdb.connect("localhost","root")
cursor = db.cursor()
cursor.execute("SELECT VERSION()")
data = cursor.fetchone()
print "Database version : %s " % data
cursor.close()
db.close()

Database version : 5.7.13 


## Create PUF URL Tracking Database

This is a database with a small number of tables used to track the PUF URLs and their JSON child URLs.

In [36]:
def create_tables(cursor):
    cursor.execute("CREATE TABLE `mrpuf` ("
        "  `id` int(11) NOT NULL AUTO_INCREMENT,"
        "  `url` varchar(255) NOT NULL,"
        "  PRIMARY KEY (`id`), UNIQUE KEY `url` (`url`)"
        ") ENGINE=InnoDB")

    cursor.execute (
        "CREATE TABLE `puftype` ("
        "  `id` int(11) NOT NULL AUTO_INCREMENT,"
        "  `type` varchar(20) NOT NULL,"
        "  PRIMARY KEY (`id`), UNIQUE KEY `type` (`type`)"
        ") ENGINE=InnoDB")

    cursor.execute (
        "CREATE TABLE `retrieval_status` ("
        "  `id` int(11) NOT NULL AUTO_INCREMENT,"
        "  `status` varchar(20) NOT NULL,"
        "  PRIMARY KEY (`id`), UNIQUE KEY `status` (`status`)"
        ") ENGINE=InnoDB")

    cursor.execute (
        "CREATE TABLE `jsonurls` ("
        "  `id` int(11) NOT NULL AUTO_INCREMENT,"
        "  `url` varchar(255) NOT NULL,"
        "  `parent_url` int(11) NOT NULL,"
        "  `puf_type` int(11) NOT NULL,"
        "  `retrieval_status` int(11) NOT NULL,"
        "  `message` varchar(1024),"
        "  PRIMARY KEY (`id`),"
        "  CONSTRAINT `jsonurls_ibfk_1` FOREIGN KEY (`puf_type`) "
        "     REFERENCES `puftype` (`id`) ON DELETE CASCADE, "
        "  CONSTRAINT `jsonurls_ibfk_2` FOREIGN KEY (`retrieval_status`) "
        "     REFERENCES `retrieval_status` (`id`) ON DELETE CASCADE, "
        "  CONSTRAINT `jsonurls_ibfk3` FOREIGN KEY (`parent_url`) "
        "     REFERENCES `mrpuf` (`id`) ON DELETE CASCADE"
        ") ENGINE=InnoDB")

In [37]:
def create_statics(cursor):
    cursor.execute("INSERT INTO puftype (type) VALUES ('PROVIDER'),('PLAN'),('FORMULARY')")
    cursor.execute("INSERT INTO retrieval_status (status) VALUES ('NEW'),('PROCESSED'),('ERROR')")

In [38]:
DB_NAME = 'mrpuf'
try:
    db = MySQLdb.connect("localhost","acauser", "password", db=DB_NAME)
    cursor = db.cursor()
    cursor.execute("DROP DATABASE IF EXISTS mrpuf".format(DB_NAME))
except OperationalError as oe:
    pass

db = MySQLdb.connect("localhost","root")
cursor = db.cursor()
cursor.execute('''CREATE DATABASE IF NOT EXISTS {0}'''.format(DB_NAME))
cursor.close()
db.close()

In [39]:
db = MySQLdb.connect("localhost","acauser", "password", db=DB_NAME)
cursor = db.cursor()
create_tables(cursor)
cursor.close()
db.close()

db = MySQLdb.connect("localhost","acauser","password", db=DB_NAME)
cursor = db.cursor()
create_statics(cursor)
cursor.close()
db.close()

In [40]:
# Test an insert into the tables
db = MySQLdb.connect("localhost","acauser", "password", db=DB_NAME)
cursor = db.cursor()

# First, insert a PUF URL
cursor.execute("INSERT INTO mrpuf (url) VALUES ('http://this.com'),('http://that.com')")

2L

In [41]:
create_statics(cursor)

In [42]:
# Next, insert a row into the jsonurl table that references the mrpuf and status and types tables
cursor.execute(
    "INSERT INTO jsonurls (url, parent_url, puf_type, retrieval_status) "
    "VALUES ('https://providers.json', "
    "(SELECT ID FROM mrpuf WHERE url='http://this.com'), "
    "(SELECT ID FROM puftype WHERE type='PROVIDER'), "
    "(SELECT ID FROM retrieval_status WHERE status='NEW'))"
)

1L

In [43]:
db = MySQLdb.connect("localhost","acauser","password", db=DB_NAME)
cursor = db.cursor()
cursor.execute("SELECT * from puftype")

0L

In [21]:
test_join(cursor)

ProgrammingError: (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'ins (url, parent_url, type, status) AS ( VALUES     ( 'https://providers.json','' at line 1")

In [121]:
cursor.execute("INSERT INTO puftype (puf_type) VALUES ('provider'),('plan'),('formulary')")

3L

In [22]:
cursor.close()
db.close()

In [18]:
def test_join(cursor):
    cursor.execute(
        "WITH ins (url, parent_url, type, status) AS "
        "( VALUES "
        "    ( 'https://providers.json','http://this.com', 'PROVIDER', 'NEW' ) ,"
        "    ( 'https://plans.json','http://that.com', 'PLAN', 'NEW' ) "
        ")"  
        "INSERT INTO jsonurls "
        "   (url, parent_url, type, status) "
        "SELECT "
        "    ins.url, pufurl.url, puftype.type, retrieval_status.status "
        "FROM "
        "  pufurl JOIN ins "
        "    ON ins.parent_url = pufurl.url "
        "UNION "
        " puftype JOIN ins "
        "    ON ins.type = putftyp.type "
        "UNION "
        " retrieval_status JOIN ins "
        "    ON ins.status = retrieval_status.status"
        )